# Обработка данных по объемам

Обработка объемных данных для выявления дней повышенной волатильности. Идея в том, чтобы найти дни в которые проходят большие объемы и есть идея направления движения.

In [82]:
import sqlite3
import pandas as pd
import numpy as np

In [83]:
# Установить соединение с базой данных
conn = sqlite3.connect(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_minute.db')

# SQL-запрос для извлечения данных
# query = "SELECT TRADEDATE, VOLUME FROM Minute"
query = "SELECT * FROM Minute"

# Загрузить данные в DataFrame
df = pd.read_sql_query(query, conn)

# Закрыть соединение
conn.close()

# Прописные в названиях колонок ---------------------------
df.columns = map(str.lower, df.columns)

df
# print(df.to_string(max_rows=10, max_cols=20))

,tradedate,secid,open,low,high,close,volume,lsttrade
0,2015-01-05 10:00:00,RIH5_2015,78450.0,74320.0,78450.0,74670.0,5970,2015-03-16
1,2015-01-05 10:01:00,RIH5_2015,74670.0,73520.0,74730.0,73580.0,3455,2015-03-16
2,2015-01-05 10:02:00,RIH5_2015,73560.0,73200.0,73570.0,73360.0,2721,2015-03-16
3,2015-01-05 10:03:00,RIH5_2015,73380.0,73250.0,74110.0,73670.0,4008,2015-03-16
4,2015-01-05 10:04:00,RIH5_2015,73650.0,73540.0,74000.0,73930.0,1471,2015-03-16
...,...,...,...,...,...,...,...,...
2029935,2024-12-17 23:45:00,RIZ4,72810.0,72810.0,72820.0,72820.0,15,2024-12-19
2029936,2024-12-17 23:46:00,RIZ4,72820.0,72820.0,72840.0,72830.0,192,2024-12-19
2029937,2024-12-17 23:47:00,RIZ4,72810.0,72800.0,72840.0,72840.0,82,2024-12-19
2029938,2024-12-17 23:48:00,RIZ4,72850.0,72790.0,72860.0,72840.0,69,2024-12-19


In [84]:
df['tradedate'] = pd.to_datetime(df['tradedate'])

# Добавление колонки с датой
df['date'] = df['tradedate'].dt.date

# Добавление колонки с временем
df['time'] = df['tradedate'].dt.time

df

,tradedate,secid,open,low,high,close,volume,lsttrade,date,time
0,2015-01-05 10:00:00,RIH5_2015,78450.0,74320.0,78450.0,74670.0,5970,2015-03-16,2015-01-05,10:00:00
1,2015-01-05 10:01:00,RIH5_2015,74670.0,73520.0,74730.0,73580.0,3455,2015-03-16,2015-01-05,10:01:00
2,2015-01-05 10:02:00,RIH5_2015,73560.0,73200.0,73570.0,73360.0,2721,2015-03-16,2015-01-05,10:02:00
3,2015-01-05 10:03:00,RIH5_2015,73380.0,73250.0,74110.0,73670.0,4008,2015-03-16,2015-01-05,10:03:00
4,2015-01-05 10:04:00,RIH5_2015,73650.0,73540.0,74000.0,73930.0,1471,2015-03-16,2015-01-05,10:04:00
...,...,...,...,...,...,...,...,...,...,...
2029935,2024-12-17 23:45:00,RIZ4,72810.0,72810.0,72820.0,72820.0,15,2024-12-19,2024-12-17,23:45:00
2029936,2024-12-17 23:46:00,RIZ4,72820.0,72820.0,72840.0,72830.0,192,2024-12-19,2024-12-17,23:46:00
2029937,2024-12-17 23:47:00,RIZ4,72810.0,72800.0,72840.0,72840.0,82,2024-12-19,2024-12-17,23:47:00
2029938,2024-12-17 23:48:00,RIZ4,72850.0,72790.0,72860.0,72840.0,69,2024-12-19,2024-12-17,23:48:00


Создаем колонку с перцентилем объема к выборке за 30 последних значений с таким же временем.

In [85]:
# Группировка по времени
df["percentile"] = (
    df.groupby("time")["volume"]
    .rolling(window=30, min_periods=1)  # Используем скользящее окно
    .apply(lambda x: (x <= x.iloc[-1]).mean() * 100)  # Рассчёт перцентиля
    .reset_index(level=0, drop=True)  # Сбрасываем лишний индекс
)

Создаем колонку со средними значениями предыдущих строк колонки percentile за каждый день

In [86]:
# Группировка по дате и расчёт среднего
df["avg_percentile"] = (
    df.groupby("date")["percentile"]  # Группировка по дате
    # .shift()  # Если текущая строка не должна учитываться при расчёте среднего (сдвиг)
    .expanding()  # Нарастающее среднее внутри каждой группы
    .mean()  # Среднее
    .reset_index(level=0, drop=True)  # Сбрасываем лишний индекс
)

In [87]:
# Округление значений в указанных колонках до двух знаков
df[["percentile", "avg_percentile"]] = df[["percentile", "avg_percentile"]].round(2)

df

,tradedate,secid,open,low,high,close,volume,lsttrade,date,time,percentile,avg_percentile
0,2015-01-05 10:00:00,RIH5_2015,78450.0,74320.0,78450.0,74670.0,5970,2015-03-16,2015-01-05,10:00:00,100.00,100.00
1,2015-01-05 10:01:00,RIH5_2015,74670.0,73520.0,74730.0,73580.0,3455,2015-03-16,2015-01-05,10:01:00,100.00,100.00
2,2015-01-05 10:02:00,RIH5_2015,73560.0,73200.0,73570.0,73360.0,2721,2015-03-16,2015-01-05,10:02:00,100.00,100.00
3,2015-01-05 10:03:00,RIH5_2015,73380.0,73250.0,74110.0,73670.0,4008,2015-03-16,2015-01-05,10:03:00,100.00,100.00
4,2015-01-05 10:04:00,RIH5_2015,73650.0,73540.0,74000.0,73930.0,1471,2015-03-16,2015-01-05,10:04:00,100.00,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2029935,2024-12-17 23:45:00,RIZ4,72810.0,72810.0,72820.0,72820.0,15,2024-12-19,2024-12-17,23:45:00,16.67,44.07
2029936,2024-12-17 23:46:00,RIZ4,72820.0,72820.0,72840.0,72830.0,192,2024-12-19,2024-12-17,23:46:00,90.00,44.12
2029937,2024-12-17 23:47:00,RIZ4,72810.0,72800.0,72840.0,72840.0,82,2024-12-19,2024-12-17,23:47:00,90.00,44.18
2029938,2024-12-17 23:48:00,RIZ4,72850.0,72790.0,72860.0,72840.0,69,2024-12-19,2024-12-17,23:48:00,76.67,44.22


In [88]:
# # Запись в Excel
# df.head(2000).to_excel('example.xlsx')

# Сохраняем первые строки в CSV
df.iloc[:3000].to_csv("head_df.csv", index=False)

# Сохраняем последние 3000 строк в CSV
df.tail(3000).to_csv("last_rows.csv", index=False)

Создаем список из значений колонки date где значение avg_percentile на 60 строке за дату больше либо равно 75.

In [89]:
# Фильтрация дат, где значение avg_percentile на 60-й строке >= 75
datetime_list = (
    df.groupby("date")  # Группировка по дате
    .nth(59)  # Выбор 60-й строки (индексация с нуля)
    .query("avg_percentile >= 75")  # Фильтр по условию
    # .index.tolist()  # Преобразование индекса в список
    .date.tolist()  # Преобразование дат в список
)

unique_values = df["date"].nunique()


# Преобразование в формат "ГГГГ-ММ-ДД" и печать
dates_list = [dt.strftime("%Y-%m-%d") for dt in datetime_list]
print(f"Список дат:, {dates_list}")
print("Длина списка:", len(dates_list))
print(f"Уникальные значения в колонке date: {unique_values}")
print(len(dates_list) / unique_values)

Список дат:, ['2015-01-05', '2015-01-26', '2015-02-06', '2015-02-09', '2015-02-12', '2015-03-26', '2015-04-09', '2015-04-10', '2015-04-13', '2015-06-22', '2015-07-08', '2015-07-28', '2015-08-03', '2015-08-24', '2015-09-01', '2016-01-18', '2016-01-19', '2016-01-20', '2016-01-21', '2016-01-22', '2016-01-25', '2016-01-29', '2016-03-17', '2016-04-13', '2016-04-18', '2016-04-20', '2016-04-21', '2016-11-09', '2016-12-12', '2017-02-28', '2017-03-09', '2017-03-10', '2017-11-08', '2018-01-18', '2018-01-25', '2018-01-26', '2018-02-06', '2018-02-26', '2018-04-09', '2018-04-10', '2018-06-19', '2018-08-09', '2018-08-10', '2018-08-13', '2018-10-11', '2018-11-26', '2019-02-14', '2019-07-12', '2019-07-25', '2019-08-02', '2020-01-21', '2020-01-23', '2020-01-29', '2020-01-30', '2020-02-03', '2020-02-10', '2020-02-26', '2020-02-27', '2020-02-28', '2020-03-02', '2020-03-06', '2020-03-12', '2020-03-13', '2020-03-20', '2020-04-16', '2020-05-21', '2020-06-11', '2020-09-24', '2021-01-28', '2021-03-01', '2021-

# Конвертирование 1 минутных котировок в 5 минутные

In [90]:
# Убедимся, что колонка  tradedate имеет тип datetime
df["tradedate"] = pd.to_datetime(df["tradedate"])

# Устанавливаем  tradedate как индекс
df.set_index("tradedate", inplace=True)

# Агрегирование до 5-минутных баров
df_m5 = df.resample("5min").agg({
    "open": "first",    # Первая цена в периоде
    "high": "max",      # Максимальная цена
    "low": "min",       # Минимальная цена
    "close": "last",    # Последняя цена
    "volume": "sum",    # Суммарный объём
})

# Сбрасываем индекс
df_m5 = df_m5.reset_index()

df_m5

,tradedate,open,high,low,close,volume
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,17625
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,7143
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,4584
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,4213
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,4498
...,...,...,...,...,...,...
1046753,2024-12-17 23:25:00,72680.0,72720.0,72680.0,72690.0,423
1046754,2024-12-17 23:30:00,72680.0,72690.0,72670.0,72690.0,69
1046755,2024-12-17 23:35:00,72700.0,72790.0,72690.0,72790.0,357
1046756,2024-12-17 23:40:00,72800.0,72820.0,72760.0,72810.0,228


### TVI

In [91]:
def calculate_ema(series, period):
    """
    Вычисляет экспоненциальное скользящее среднее (EMA) для указанного ряда данных.
    """
    return series.ewm(span=period, adjust=False).mean()

def add_tvi_column(df, r=12, s=12, u=5, point=0.0001):
    """
    Добавляет колонку TVI в DataFrame.
    
    :param df: DataFrame с колонками ['open', 'close', 'volume']
    :param r: Период для EMA для UpTicks и DownTicks
    :param s: Период для второго EMA
    :param u: Период для EMA на TVI_calculate
    :param point: Минимальное изменение цены (параметр MyPoint в MQL4)
    :return: DataFrame с добавленной колонкой 'TVI'
    """
    # Проверяем, что необходимые колонки существуют
    required_columns = ['open', 'close', 'volume']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"DataFrame должен содержать колонку '{col}'")

    # Вычисляем UpTicks и DownTicks
    df['UpTicks'] = (df['volume'] + (df['close'] - df['open']) / point) / 2
    df['DownTicks'] = df['volume'] - df['UpTicks']

    # EMA для UpTicks и DownTicks
    df['EMA_UpTicks'] = calculate_ema(df['UpTicks'], r)
    df['EMA_DownTicks'] = calculate_ema(df['DownTicks'], r)

    # Второе EMA для UpTicks и DownTicks
    df['DEMA_UpTicks'] = calculate_ema(df['EMA_UpTicks'], s)
    df['DEMA_DownTicks'] = calculate_ema(df['EMA_DownTicks'], s)

    # Расчет TVI_calculate
    df['TVI_calculate'] = 100.0 * (df['DEMA_UpTicks'] - df['DEMA_DownTicks']) / (
        df['DEMA_UpTicks'] + df['DEMA_DownTicks']
    )

    # EMA на TVI_calculate
    df['TVI'] = calculate_ema(df['TVI_calculate'], u)

    # Удаляем промежуточные колонки, если они не нужны
    df.drop(columns=['UpTicks', 'DownTicks', 'EMA_UpTicks', 'EMA_DownTicks', 
                     'DEMA_UpTicks', 'DEMA_DownTicks', 'TVI_calculate'], inplace=True)

    return df

# Добавляем колонку TVI
df_m5 = add_tvi_column(df_m5, r=12, s=12, u=5, point=0.1)

# Результат
df_m5

,tradedate,open,high,low,close,volume,TVI
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,17625,-256.453901
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,7143,-255.254435
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,4584,-253.367957
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,4213,-250.544546
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,4498,-246.875928
...,...,...,...,...,...,...,...
1046753,2024-12-17 23:25:00,72680.0,72720.0,72680.0,72690.0,423,91.925743
1046754,2024-12-17 23:30:00,72680.0,72690.0,72670.0,72690.0,69,94.212201
1046755,2024-12-17 23:35:00,72700.0,72790.0,72690.0,72790.0,357,99.302020
1046756,2024-12-17 23:40:00,72800.0,72820.0,72760.0,72810.0,228,104.138889


## GannHilo

In [92]:
def calculate_sma(series, period):
    """
    Вычисляет простую скользящую среднюю (SMA).
    """
    return series.rolling(window=period, min_periods=1).mean()

def add_gann_hilo_column(df, period=10):
    """
    Добавляет индикатор GannHiLo как колонки в DataFrame.
    
    :param df: DataFrame с колонками ['close', 'high', 'low']
    :param period: Период для расчета SMA
    :return: DataFrame с добавленными колонками 'Gann_HiLo', 'Gann_Up', 'Gann_Down'
    """
    # Проверка на необходимые колонки
    required_columns = ['close', 'high', 'low']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"DataFrame должен содержать колонку '{col}'")

    # Расчет SMA для high и low
    df['SMA_High'] = calculate_sma(df['high'], period)
    df['SMA_Low'] = calculate_sma(df['low'], period)

    # Инициализация GannHiLo
    df['Gann_HiLo'] = 0  # 1 для UpTrend, -1 для DownTrend
    df['Gann_Up'] = 0    # 1 для Up
    df['Gann_Down'] = 0  # 1 для Down

    # Логика GannHiLo
    for i in range(len(df)):
        if i == 0:
            continue  # Пропускаем первый индекс, так как нет данных для сравнения

        if df['close'].iloc[i] > df['SMA_High'].iloc[i - 1]:
            df.at[i, 'Gann_HiLo'] = 1
        elif df['close'].iloc[i] < df['SMA_Low'].iloc[i - 1]:
            df.at[i, 'Gann_HiLo'] = -1

        # Заполняем Up и Down
        if df['Gann_HiLo'].iloc[i] == 1:
            df.at[i, 'Gann_Up'] = 1
            df.at[i, 'Gann_Down'] = 0
        elif df['Gann_HiLo'].iloc[i] == -1:
            df.at[i, 'Gann_Down'] = 1
            df.at[i, 'Gann_Up'] = 0

    # Удаление промежуточных колонок, если они не нужны
    df.drop(columns=['SMA_High', 'SMA_Low', 'Gann_Up', 'Gann_Down'], inplace=True)

    return df

# Добавляем GannHiLo индикатор
df_m5 = add_gann_hilo_column(df_m5, period=10)

# Результат
df_m5


,tradedate,open,high,low,close,volume,TVI,Gann_HiLo
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,17625,-256.453901,0
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,7143,-255.254435,0
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,4584,-253.367957,0
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,4213,-250.544546,0
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,4498,-246.875928,0
...,...,...,...,...,...,...,...,...
1046753,2024-12-17 23:25:00,72680.0,72720.0,72680.0,72690.0,423,91.925743,1
1046754,2024-12-17 23:30:00,72680.0,72690.0,72670.0,72690.0,69,94.212201,1
1046755,2024-12-17 23:35:00,72700.0,72790.0,72690.0,72790.0,357,99.302020,1
1046756,2024-12-17 23:40:00,72800.0,72820.0,72760.0,72810.0,228,104.138889,1


## T3 (T3ma)

In [93]:
# def calculate_t3ma(df, period=8, b=0.618):
#     """
#     Рассчитывает индикатор T3MA и добавляет его как новую колонку в DataFrame.

#     :param df: DataFrame с колонкой 'close'
#     :param period: Период T3MA
#     :param b: Параметр T3MA (значение сглаживания)
#     :return: DataFrame с добавленной колонкой 'T3MA'
#     """
#     # Проверка на наличие колонки 'close'
#     if 'close' not in df.columns:
#         raise ValueError("DataFrame должен содержать колонку 'close'")

#     # Коэффициенты
#     b2 = b ** 2
#     b3 = b2 * b
#     c1 = -b3
#     c2 = 3 * (b2 + b3)
#     c3 = -3 * (2 * b2 + b + b3)
#     c4 = 1 + 3 * b + b3 + 3 * b2
#     n = max(1, 1 + 0.5 * (period - 1))
#     w1 = 2 / (n + 1)
#     w2 = 1 - w1

#     # Инициализация временных массивов
#     e1 = np.zeros(len(df))
#     e2 = np.zeros(len(df))
#     e3 = np.zeros(len(df))
#     e4 = np.zeros(len(df))
#     e5 = np.zeros(len(df))
#     e6 = np.zeros(len(df))
#     t3 = np.zeros(len(df))

#     # Вычисления T3MA
#     for i in range(len(df)):
#         close = df['close'].iloc[i]
#         if i == 0:
#             e1[i] = close
#             e2[i] = close
#             e3[i] = close
#             e4[i] = close
#             e5[i] = close
#             e6[i] = close
#         else:
#             e1[i] = w1 * close + w2 * e1[i - 1]
#             e2[i] = w1 * e1[i] + w2 * e2[i - 1]
#             e3[i] = w1 * e2[i] + w2 * e3[i - 1]
#             e4[i] = w1 * e3[i] + w2 * e4[i - 1]
#             e5[i] = w1 * e4[i] + w2 * e5[i - 1]
#             e6[i] = w1 * e5[i] + w2 * e6[i - 1]

#         t3[i] = c1 * e6[i] + c2 * e5[i] + c3 * e4[i] + c4 * e3[i]

#     # Добавление колонки T3MA в DataFrame
#     df['T3MA'] = t3
#     return df

# # Рассчитываем T3MA и добавляем в DataFrame
# df_m5 = calculate_t3ma(df_m5, period=8, b=0.618)

# # Результат
# df_m5

In [95]:
def calculate_t3ma(df, period=8, b=0.618):
    """
    Рассчитывает индикатор T3MA и добавляет его как новую колонку в DataFrame.

    :param df: DataFrame с колонкой 'close'
    :param period: Период T3MA
    :param b: Параметр T3MA (значение сглаживания)
    :return: DataFrame с добавленной колонкой 'T3MA'
    """
    # Проверка на наличие колонки 'close'
    if 'close' not in df.columns:
        raise ValueError("DataFrame должен содержать колонку 'close'")

    # Коэффициенты
    b2 = b ** 2
    b3 = b2 * b
    c1 = -b3
    c2 = 3 * (b2 + b3)
    c3 = -3 * (2 * b2 + b + b3)
    c4 = 1 + 3 * b + b3 + 3 * b2
    n = max(1, 1 + 0.5 * (period - 1))
    w1 = 2 / (n + 1)
    w2 = 1 - w1

    # Инициализация временных массивов
    e1 = np.zeros(len(df))
    e2 = np.zeros(len(df))
    e3 = np.zeros(len(df))
    e4 = np.zeros(len(df))
    e5 = np.zeros(len(df))
    e6 = np.zeros(len(df))
    t3 = np.zeros(len(df))

    # Вычисления T3MA
    for i in range(len(df)):
        close = df['close'].iloc[i]
        
        # Для первого значения используем само закрытие
        if i == 0:
            e1[i] = close
            e2[i] = close
            e3[i] = close
            e4[i] = close
            e5[i] = close
            e6[i] = close
        else:
            # Для последующих значений вычисляем сглаженные EMA
            e1[i] = w1 * close + w2 * e1[i - 1]
            e2[i] = w1 * e1[i] + w2 * e2[i - 1]
            e3[i] = w1 * e2[i] + w2 * e3[i - 1]
            e4[i] = w1 * e3[i] + w2 * e4[i - 1]
            e5[i] = w1 * e4[i] + w2 * e5[i - 1]
            e6[i] = w1 * e5[i] + w2 * e6[i - 1]

        # Вычисление финального значения T3MA
        t3[i] = c1 * e6[i] + c2 * e5[i] + c3 * e4[i] + c4 * e3[i]

    # Добавление колонки T3MA в DataFrame
    df['T3MA'] = t3
    return df

# Рассчитываем T3MA и добавляем в DataFrame
df_m5 = calculate_t3ma(df_m5, period=8, b=0.618)

# Результат
# df_m5
print(df_m5.to_string(max_rows=100, max_cols=20))

                  tradedate     open     high      low    close  volume         TVI  Gann_HiLo          T3MA
0       2015-01-05 10:00:00  78450.0  78450.0  73200.0  73930.0   17625 -256.453901          0  73930.000000
1       2015-01-05 10:05:00  73890.0  74700.0  73660.0  74700.0    7143 -255.254435          0  74030.138493
2       2015-01-05 10:10:00  74700.0  74720.0  73830.0  73950.0    4584 -253.367957          0  74092.939249
3       2015-01-05 10:15:00  73970.0  74440.0  73620.0  74430.0    4213 -250.544546          0  74167.176629
4       2015-01-05 10:20:00  74400.0  74720.0  74220.0  74410.0    4498 -246.875928          0  74244.053078
5       2015-01-05 10:25:00  74420.0  74420.0  73600.0  73610.0    4670 -242.853373         -1  74207.350215
6       2015-01-05 10:30:00  73630.0  73910.0  73320.0  73490.0    3418 -238.535213         -1  74076.673033
7       2015-01-05 10:35:00  73460.0  73910.0  73420.0  73720.0    1694 -233.936716          0  73942.053757
8       2015-01-05 